In [4]:
# Modelo de algoritmo de correção de erros adaptativo

import os 
import pandas as pd
import pickle
import re
from operator import itemgetter
import random

# Abrir arquivo de dados

a_file = open("data.pkl", "rb")
data = pickle.load(a_file)

# Parametros de configuração da simulação

modulacao = 'BPSK'

sinalRuido = []
for i in range(0, 3):
    sinalRuido.append(round(random.uniform(3, 12), 1))
print('Vetor SNR:',sinalRuido, '\n\n')

codecRate = 64 # Kbps
codecPayload = 160  # Bytes
pps = 50 # milisegundos
cabecalhoIP = 40 # Bytes 
quantChamadas = 2
bandaDisponivel = 170 # Kbps

availableConfigs = []
results = []

for mod, param in data.items():
    if mod == modulacao:
        for snr in sinalRuido:
            if snr in param.keys():
                for tupla in param.get(snr):
                    rate = tupla[3]
                    if quantChamadas > 1:
                        bandaRequerida = ((((codecPayload * rate + cabecalhoIP) * 8) * pps) / 1000) * quantChamadas
                        if bandaRequerida <= bandaDisponivel:
                            availableConfigs.append(tupla)
                    else:
                        bandaRequerida = (((codecPayload * rate + cabecalhoIP) * 8) * pps) / 1000
                        if bandaRequerida <= bandaDisponivel:
                            availableConfigs.append(tupla)

                print('A banda disponível é de:', bandaDisponivel, 'Kbps')

                tabela = pd.DataFrame(availableConfigs, columns =['MOS', 'BER', 'FEC', 'RATE']) 
                print('Configurações possiveis para modulação', mod, 'e SNR', snr, 'dB:' '\n')
                print(tabela, '\n')

                try:
                    value = sorted(sorted(availableConfigs, key=lambda element: element[3]), 
                                   key=lambda element: element[0], reverse=True)[0]

#   Salva resultados dos testes em lista

                    results.append(value)

                    print('Melhor configuração: ', value[2])
                    print('MOS=', value[0])
                    print('---------------------------------------------------------', '\n')
                except:
                    print('Impossivel: Largura de banda insuficiente')

                list.clear(availableConfigs)
# Fechar arquivo

a_file.close()

Vetor SNR: [10.6, 3.1, 4.5] 


A banda disponível é de: 170 Kbps
Configurações possiveis para modulação BPSK e SNR 10.6 dB:

      MOS           BER           FEC      RATE
0  4.3394  8.056800e-07       No Code  1.000000
1  4.5000  0.000000e+00  RS(255, 247)  1.032389 

Melhor configuração:  RS(255, 247)
MOS= 4.5
--------------------------------------------------------- 

A banda disponível é de: 170 Kbps
Configurações possiveis para modulação BPSK e SNR 3.1 dB:

      MOS       BER           FEC      RATE
0  1.0338  0.021640       No Code  1.000000
1  1.0887  0.021934  RS(255, 247)  1.032389 

Melhor configuração:  RS(255, 247)
MOS= 1.0887
--------------------------------------------------------- 

A banda disponível é de: 170 Kbps
Configurações possiveis para modulação BPSK e SNR 4.5 dB:

      MOS       BER           FEC      RATE
0  1.0353  0.008800       No Code  1.000000
1  1.0317  0.009088  RS(255, 247)  1.032389 

Melhor configuração:  No Code
MOS= 1.0353
----------------------